In [1]:
%cd ../../

/home/PJWSTK/s14028/engineer/Practical


In [2]:
import keras
import scipy.io as mat

from common import *
from augmentation import add_pmap
from augmentation import augmentation_data
from augmentation import augment_data

from CNN.CNN_v9 import CNN_v9

Using TensorFlow backend.


In [3]:
perspective = mat.loadmat("mall_dataset/perspective_roi.mat")["pMapN"]

perspective /= np.min(perspective)
perspective = np.round(perspective).astype(np.uint8)

train, test = data_sets()
image_tensors = train[0], test[0]
person_coo_tensors = train[1], test[1]
count_matrix = train[2], test[2]

image_train, image_test = image_tensors
person_coo_train, person_coo_test = person_coo_tensors
count_train, count_test = count_matrix
count_train = count_train.astype(np.uint16)
count_test = count_test.astype(np.uint16)

image_train = add_pmap(image_train, perspective)
image_test = add_pmap(image_test, perspective)

In [4]:
cnn_v9 = CNN_v9((480, 640, 4), split_into_parts=20)
cnn_v9.def_model()

In [5]:
images = cnn_v9._prepare_images(image_train)
anwsers = cnn_v9._prepare_anwsers(person_coo_train)

In [6]:
ones_count = (np.sum(anwsers == 1) * 0.25).astype(np.uint32)
zeros_count = (ones_count / 0.25 * 0.75).astype(np.uint32)
validation_length = (zeros_count + ones_count).astype(np.int32)

val_indices = np.concatenate([np.where(anwsers == 0)[0][:zeros_count],
                              np.where(anwsers == 1)[0][:ones_count]])

In [7]:
anwsers[val_indices[zeros_count:]] = 0

In [8]:
val_indices = -(images.shape[0] - val_indices)

In [9]:
generator = keras.preprocessing.image.ImageDataGenerator(rotation_range=20)

augmentation = augmentation_data(image_train, anwsers, 20)
augmented_data = augment_data(generator, augmentation, images, anwsers)
images, anwsers = augmented_data

In [10]:
anwsers[val_indices[zeros_count:]] = 1

images[-validation_length:], images[val_indices] = images[val_indices], images[-validation_length:]
anwsers[-validation_length:], anwsers[val_indices] = anwsers[val_indices], anwsers[-validation_length:]

In [11]:
parallel_model = keras.utils.multi_gpu_model(
        cnn_v9.model,
        gpus=2,
        cpu_merge=False)
parallel_model.compile(
        loss="binary_crossentropy", 
        optimizer="adam", 
        metrics=["accuracy"])

In [12]:
path = "CNN/CNN_v9/weights/cnn_v9_1e_6_{}"

class ModelSaver(keras.callbacks.Callback):
    __path: str
    __current_epoch: int
        
    def __init__(self, path: str, epoch: int = 0):
        self.__path = path
        self.__current_epoch = epoch
        
    def on_epoch_end(self, *args, **kwargs):
        epoch_path = self.__path.format(self.__current_epoch)
        self.__current_epoch += 1
        
        parallel_model.save_weights(epoch_path)

model_saver = ModelSaver(path, 5)

In [13]:
parallel_model.optimizer.lr.assign(1e-4)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=1e-04>

In [14]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=5000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 180s 145us/step - loss: 0.7217 - accuracy: 0.9025 - val_loss: 0.5272 - val_accuracy: 0.9034
Epoch 2/5
1244460/1244460 [==============================] - 176s 141us/step - loss: 0.2338 - accuracy: 0.9604 - val_loss: 0.3163 - val_accuracy: 0.8966
Epoch 3/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.1613 - accuracy: 0.9690 - val_loss: 0.2768 - val_accuracy: 0.9035
Epoch 4/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.1318 - accuracy: 0.9728 - val_loss: 0.2436 - val_accuracy: 0.9139
Epoch 5/5
1244460/1244460 [==============================] - 175s 141us/step - loss: 0.1143 - accuracy: 0.9751 - val_loss: 0.2380 - val_accuracy: 0.9128


In [15]:
parallel_model.optimizer.lr.assign(1e-5)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=1e-05>

In [16]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=5000, epochs=10,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/10
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.0988 - accuracy: 0.9792 - val_loss: 0.2128 - val_accuracy: 0.9245
Epoch 2/10
1244460/1244460 [==============================] - 176s 141us/step - loss: 0.0955 - accuracy: 0.9801 - val_loss: 0.2103 - val_accuracy: 0.9265
Epoch 3/10
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0936 - accuracy: 0.9805 - val_loss: 0.2110 - val_accuracy: 0.9248
Epoch 4/10
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0917 - accuracy: 0.9809 - val_loss: 0.2053 - val_accuracy: 0.9272
Epoch 5/10
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0902 - accuracy: 0.9811 - val_loss: 0.2010 - val_accuracy: 0.9291
Epoch 6/10
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0883 - accuracy: 0.9816 - val_loss: 0.2033 - val_accuracy: 0.9278
Epoch 7/10
1244460/12444

In [17]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=5000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 176s 142us/step - loss: 0.0802 - accuracy: 0.9832 - val_loss: 0.1850 - val_accuracy: 0.9346
Epoch 2/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.0786 - accuracy: 0.9835 - val_loss: 0.1817 - val_accuracy: 0.9352
Epoch 3/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.0770 - accuracy: 0.9838 - val_loss: 0.1810 - val_accuracy: 0.9357
Epoch 4/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.0756 - accuracy: 0.9840 - val_loss: 0.1781 - val_accuracy: 0.9366
Epoch 5/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.0742 - accuracy: 0.9844 - val_loss: 0.1746 - val_accuracy: 0.9377


In [18]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=5000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0729 - accuracy: 0.9846 - val_loss: 0.1696 - val_accuracy: 0.9403
Epoch 2/5
1244460/1244460 [==============================] - 175s 141us/step - loss: 0.0717 - accuracy: 0.9849 - val_loss: 0.1649 - val_accuracy: 0.9427
Epoch 3/5
1244460/1244460 [==============================] - 175s 141us/step - loss: 0.0703 - accuracy: 0.9852 - val_loss: 0.1656 - val_accuracy: 0.9411
Epoch 4/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.0688 - accuracy: 0.9856 - val_loss: 0.1618 - val_accuracy: 0.9432
Epoch 5/5
1244460/1244460 [==============================] - 175s 141us/step - loss: 0.0678 - accuracy: 0.9857 - val_loss: 0.1562 - val_accuracy: 0.9466


In [19]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=5000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 175s 141us/step - loss: 0.0668 - accuracy: 0.9860 - val_loss: 0.1579 - val_accuracy: 0.9444
Epoch 2/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0654 - accuracy: 0.9864 - val_loss: 0.1571 - val_accuracy: 0.9445
Epoch 3/5
1244460/1244460 [==============================] - 176s 141us/step - loss: 0.0643 - accuracy: 0.9866 - val_loss: 0.1513 - val_accuracy: 0.9477
Epoch 4/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0633 - accuracy: 0.9869 - val_loss: 0.1480 - val_accuracy: 0.9501
Epoch 5/5
1244460/1244460 [==============================] - 177s 142us/step - loss: 0.0622 - accuracy: 0.9870 - val_loss: 0.1431 - val_accuracy: 0.9514


In [20]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=5000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 175s 141us/step - loss: 0.0613 - accuracy: 0.9872 - val_loss: 0.1439 - val_accuracy: 0.9517
Epoch 2/5
1244460/1244460 [==============================] - 177s 142us/step - loss: 0.0602 - accuracy: 0.9876 - val_loss: 0.1413 - val_accuracy: 0.9528
Epoch 3/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.0593 - accuracy: 0.9876 - val_loss: 0.1363 - val_accuracy: 0.9562
Epoch 4/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0584 - accuracy: 0.9880 - val_loss: 0.1374 - val_accuracy: 0.9546
Epoch 5/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0572 - accuracy: 0.9883 - val_loss: 0.1340 - val_accuracy: 0.9561


In [21]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=5000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0564 - accuracy: 0.9885 - val_loss: 0.1333 - val_accuracy: 0.9558
Epoch 2/5
1244460/1244460 [==============================] - 175s 141us/step - loss: 0.0557 - accuracy: 0.9886 - val_loss: 0.1277 - val_accuracy: 0.9592
Epoch 3/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0548 - accuracy: 0.9888 - val_loss: 0.1294 - val_accuracy: 0.9574
Epoch 4/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0539 - accuracy: 0.9891 - val_loss: 0.1308 - val_accuracy: 0.9571
Epoch 5/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0532 - accuracy: 0.9893 - val_loss: 0.1236 - val_accuracy: 0.9610


In [22]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=5000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0525 - accuracy: 0.9894 - val_loss: 0.1241 - val_accuracy: 0.9596
Epoch 2/5
1244460/1244460 [==============================] - 173s 139us/step - loss: 0.0515 - accuracy: 0.9896 - val_loss: 0.1196 - val_accuracy: 0.9623
Epoch 3/5
1244460/1244460 [==============================] - 173s 139us/step - loss: 0.0510 - accuracy: 0.9898 - val_loss: 0.1190 - val_accuracy: 0.9620
Epoch 4/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0501 - accuracy: 0.9900 - val_loss: 0.1145 - val_accuracy: 0.9649
Epoch 5/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0496 - accuracy: 0.9901 - val_loss: 0.1135 - val_accuracy: 0.9647


In [23]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=5000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 176s 141us/step - loss: 0.0486 - accuracy: 0.9904 - val_loss: 0.1094 - val_accuracy: 0.9670
Epoch 2/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0480 - accuracy: 0.9905 - val_loss: 0.1091 - val_accuracy: 0.9675
Epoch 3/5
1244460/1244460 [==============================] - 173s 139us/step - loss: 0.0475 - accuracy: 0.9906 - val_loss: 0.1082 - val_accuracy: 0.9686
Epoch 4/5
1244460/1244460 [==============================] - 173s 139us/step - loss: 0.0468 - accuracy: 0.9908 - val_loss: 0.1075 - val_accuracy: 0.9673
Epoch 5/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0460 - accuracy: 0.9910 - val_loss: 0.1020 - val_accuracy: 0.9703


In [24]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=5000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0456 - accuracy: 0.9911 - val_loss: 0.1013 - val_accuracy: 0.9705
Epoch 2/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0449 - accuracy: 0.9912 - val_loss: 0.0992 - val_accuracy: 0.9714
Epoch 3/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.0443 - accuracy: 0.9914 - val_loss: 0.0962 - val_accuracy: 0.9727
Epoch 4/5
1244460/1244460 [==============================] - 175s 140us/step - loss: 0.0437 - accuracy: 0.9916 - val_loss: 0.0940 - val_accuracy: 0.9739
Epoch 5/5
1244460/1244460 [==============================] - 174s 139us/step - loss: 0.0431 - accuracy: 0.9917 - val_loss: 0.0941 - val_accuracy: 0.9739


In [25]:
parallel_model.optimizer.lr.assign(1e-6)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=1e-06>

In [26]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=5000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 172s 138us/step - loss: 0.0413 - accuracy: 0.9925 - val_loss: 0.0916 - val_accuracy: 0.9754
Epoch 2/5
1244460/1244460 [==============================] - 172s 138us/step - loss: 0.0410 - accuracy: 0.9926 - val_loss: 0.0907 - val_accuracy: 0.9760
Epoch 3/5
1244460/1244460 [==============================] - 173s 139us/step - loss: 0.0408 - accuracy: 0.9927 - val_loss: 0.0902 - val_accuracy: 0.9762
Epoch 4/5
1244460/1244460 [==============================] - 174s 139us/step - loss: 0.0406 - accuracy: 0.9928 - val_loss: 0.0899 - val_accuracy: 0.9765
Epoch 5/5
1244460/1244460 [==============================] - 173s 139us/step - loss: 0.0406 - accuracy: 0.9928 - val_loss: 0.0901 - val_accuracy: 0.9766


In [27]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=5000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0405 - accuracy: 0.9928 - val_loss: 0.0890 - val_accuracy: 0.9769
Epoch 2/5
1244460/1244460 [==============================] - 174s 139us/step - loss: 0.0404 - accuracy: 0.9928 - val_loss: 0.0898 - val_accuracy: 0.9765
Epoch 3/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0403 - accuracy: 0.9929 - val_loss: 0.0884 - val_accuracy: 0.9772
Epoch 4/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0403 - accuracy: 0.9929 - val_loss: 0.0886 - val_accuracy: 0.9772
Epoch 5/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0403 - accuracy: 0.9928 - val_loss: 0.0881 - val_accuracy: 0.9776


In [28]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=5000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 174s 139us/step - loss: 0.0400 - accuracy: 0.9930 - val_loss: 0.0887 - val_accuracy: 0.9772
Epoch 2/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0400 - accuracy: 0.9929 - val_loss: 0.0879 - val_accuracy: 0.9774
Epoch 3/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0399 - accuracy: 0.9930 - val_loss: 0.0879 - val_accuracy: 0.9778
Epoch 4/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0398 - accuracy: 0.9931 - val_loss: 0.0875 - val_accuracy: 0.9778
Epoch 5/5
1244460/1244460 [==============================] - 173s 139us/step - loss: 0.0399 - accuracy: 0.9930 - val_loss: 0.0870 - val_accuracy: 0.9781


In [29]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=5000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 173s 139us/step - loss: 0.0397 - accuracy: 0.9930 - val_loss: 0.0868 - val_accuracy: 0.9780
Epoch 2/5
1244460/1244460 [==============================] - 173s 139us/step - loss: 0.0397 - accuracy: 0.9931 - val_loss: 0.0863 - val_accuracy: 0.9784
Epoch 3/5
1244460/1244460 [==============================] - 173s 139us/step - loss: 0.0395 - accuracy: 0.9931 - val_loss: 0.0866 - val_accuracy: 0.9783
Epoch 4/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0394 - accuracy: 0.9932 - val_loss: 0.0869 - val_accuracy: 0.9778
Epoch 5/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0395 - accuracy: 0.9931 - val_loss: 0.0864 - val_accuracy: 0.9781


In [30]:
parallel_model.optimizer.lr.assign(1e-5)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=1e-05>

In [31]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=5000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0413 - accuracy: 0.9922 - val_loss: 0.0923 - val_accuracy: 0.9745
Epoch 2/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0407 - accuracy: 0.9923 - val_loss: 0.0871 - val_accuracy: 0.9773
Epoch 3/5
1244460/1244460 [==============================] - 174s 139us/step - loss: 0.0403 - accuracy: 0.9924 - val_loss: 0.0851 - val_accuracy: 0.9775
Epoch 4/5
1244460/1244460 [==============================] - 173s 139us/step - loss: 0.0398 - accuracy: 0.9925 - val_loss: 0.0846 - val_accuracy: 0.9782
Epoch 5/5
1244460/1244460 [==============================] - 173s 139us/step - loss: 0.0391 - accuracy: 0.9927 - val_loss: 0.0841 - val_accuracy: 0.9781


In [32]:
parallel_model.optimizer.lr.assign(1e-6)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=1e-06>

In [33]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=5000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 173s 139us/step - loss: 0.0374 - accuracy: 0.9933 - val_loss: 0.0797 - val_accuracy: 0.9806
Epoch 2/5
1244460/1244460 [==============================] - 174s 139us/step - loss: 0.0371 - accuracy: 0.9936 - val_loss: 0.0791 - val_accuracy: 0.9808
Epoch 3/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0368 - accuracy: 0.9937 - val_loss: 0.0783 - val_accuracy: 0.9814
Epoch 4/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0367 - accuracy: 0.9936 - val_loss: 0.0782 - val_accuracy: 0.9813
Epoch 5/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0367 - accuracy: 0.9937 - val_loss: 0.0780 - val_accuracy: 0.9817


In [34]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=5000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0367 - accuracy: 0.9937 - val_loss: 0.0776 - val_accuracy: 0.9815
Epoch 2/5
1244460/1244460 [==============================] - 174s 139us/step - loss: 0.0364 - accuracy: 0.9939 - val_loss: 0.0767 - val_accuracy: 0.9823
Epoch 3/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0364 - accuracy: 0.9938 - val_loss: 0.0775 - val_accuracy: 0.9816
Epoch 4/5
1244460/1244460 [==============================] - 174s 139us/step - loss: 0.0363 - accuracy: 0.9938 - val_loss: 0.0763 - val_accuracy: 0.9825
Epoch 5/5
1244460/1244460 [==============================] - 173s 139us/step - loss: 0.0362 - accuracy: 0.9939 - val_loss: 0.0771 - val_accuracy: 0.9821


In [35]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=5000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0361 - accuracy: 0.9939 - val_loss: 0.0763 - val_accuracy: 0.9820
Epoch 2/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0361 - accuracy: 0.9939 - val_loss: 0.0758 - val_accuracy: 0.9827
Epoch 3/5
1244460/1244460 [==============================] - 174s 139us/step - loss: 0.0361 - accuracy: 0.9939 - val_loss: 0.0759 - val_accuracy: 0.9822
Epoch 4/5
1244460/1244460 [==============================] - 173s 139us/step - loss: 0.0360 - accuracy: 0.9938 - val_loss: 0.0759 - val_accuracy: 0.9824
Epoch 5/5
1244460/1244460 [==============================] - 174s 140us/step - loss: 0.0359 - accuracy: 0.9940 - val_loss: 0.0754 - val_accuracy: 0.9825
